# Introduction to Quantum Computing using PyQuil

*make sure that qvm and quilc are running in server mode

Quantum programs are written in Forest using the `Program` object. This `Program` abstraction will help us compose Quil programs.

In [1]:
from pyquil import Program

#### Quantum states are complex vectors on the Bloch sphere, and quantum operations are matrices with two properties:

They are reversible.
When applied to a state vector on the Bloch sphere, the resulting vector is also on the Bloch sphere.
Matrices that satisfy these two properties are called unitary matrices. Such matrices have the characteristic property that their complex conjugate transpose is equal to their inverse, a property directly linked to the requirement that the probabilities of measuring qubits in any of the allowed states must sum to 1. Applying an operation to a quantum state is the same as multiplying a vector by one of these matrices. Such an operation is called a gate.

Since individual qubits are two-dimensional vectors, operations on individual qubits are 2x2 matrices. The identity matrix leaves the state vector unchanged:

$$
I = \left(\begin{matrix}
1 & 0\\
0 & 1
\end{matrix}\right)
$$
so the a program that applies this operation to the zero state is just
$$ I\,|\,0\rangle = \left(\begin{matrix}
1 & 0\\
0 & 1
\end{matrix}\right)\left(\begin{matrix}
1 \\
0 
\end{matrix}\right) = \left(\begin{matrix}
1 \\
0 
\end{matrix}\right) = |\,0\rangle$$

Other standard gates on single qubits are given by the Pauli operator matrices
$$
X = \left(\begin{matrix}
0 & 1\\
1 & 0
\end{matrix}\right)
\qquad
Y = \left(\begin{matrix}
0 & -i\\
i & 0
\end{matrix}\right)
\qquad
Z = \left(\begin{matrix}
1 & 0\\
0 & -1
\end{matrix}\right)
$$

A special single qubit operation is the Hadamard gate.

$$
H = \frac{1}{\sqrt{2}}\left(\begin{matrix}
1 & 1\\
1 & -1
\end{matrix}\right)
$$

Operations can also be applied to composite states of multiple qubits. One common example is the controlled-not or CNOT gate that works on two qubits.  Its matrix form is:
$$
CNOT = \left(\begin{matrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
\end{matrix}\right)
$$

Programs are constructed by adding these quantum gates to it, which are defined in the `gates` module. We can import all standard gates with the following:

In [2]:
from pyquil.gates import *

Let’s instantiate a `Program` and add an operation to it. We will act an `X` gate on qubit 0.

In [3]:
p = Program()
p += X(0)

All qubits begin in the ground state. This means that if we measure a qubit without applying operations on it, we expect to receive a 0 result. The `X` gate will rotate qubit 0 from the ground state to the excited state, so a measurement immediately after should return a 1 result.

In [4]:
from pyquil.api import WavefunctionSimulator

wavefunction_simulator = WavefunctionSimulator()

# basic one: the identity operation, called I. I takes one argument, the index
# of the qubit that it should be applied to.
# So let's make a quantum program that allocates one qubit (qubit #0) and does nothing to it
p = Program(I(0))

# quantum states are called wavefunctions for historical reasons
# so we can run this basic program on our connection to the simulator.
# This call will return the state of our qubits after we run program p.
print(wavefunction_simulator.wavefunction(p))

(1+0j)|0>


In [44]:
# we can import the qubit "flip" operation (called X), which we'll talk about in the next section
# and see what that does.
p = Program(X(0))

alpha, beta = wavefunction_simulator.wavefunction(p)
print("Our qubit is in the state alpha={} and beta={}".format(alpha, beta))

Our qubit is in the state alpha=0j and beta=(1+0j)


# Multiple qubits also produce the expected scaling of the state

In [45]:
p = Program(I(0), I(1))
print(p,"The quantum state is of dimension:", len(wavefunction_simulator.wavefunction(p)))
print()

p = Program(I(0), I(1), I(2), I(3))
print(p,"The quantum state is of dimension:", len(wavefunction_simulator.wavefunction(p)))
print()

p = Program()
for x in range(10):
    p.inst(I(x))
print(p,"The quantum state is of dimension:", len(wavefunction_simulator.wavefunction(p)))

I 0
I 1
 The quantum state is of dimension: 2

I 0
I 1
I 2
I 3
 The quantum state is of dimension: 4

I 0
I 1
I 2
I 3
I 4
I 5
I 6
I 7
I 8
I 9
 The quantum state is of dimension: 10


# Exploring gates with WavefunctionSimulator

Lets visualize the following gates:
$$
X = \left(\begin{matrix}
0 & 1\\
1 & 0
\end{matrix}\right)
\qquad
Y = \left(\begin{matrix}
0 & -i\\
i & 0
\end{matrix}\right)
\qquad
Z = \left(\begin{matrix}
1 & 0\\
0 & -1
\end{matrix}\right)
\qquad
H = \frac{1}{\sqrt{2}}\left(\begin{matrix}
1 & 1\\
1 & -1
\end{matrix}\right)
$$

In [5]:
p = Program(X(0))
wavefunction = wavefunction_simulator.wavefunction(p)
print("X|0> = ", wavefunction)
print("The outcome probabilities are", wavefunction.get_outcome_probs())
print("This looks like a bit flip.\n")

p = Program(Y(0))
wavefunction = wavefunction_simulator.wavefunction(p)
print("Y|0> = ", wavefunction)
print("The outcome probabilities are", wavefunction.get_outcome_probs())
print("This also looks like a bit flip.\n")

p = Program(Z(0))
wavefunction = wavefunction_simulator.wavefunction(p)
print("Z|0> = ", wavefunction)
print("The outcome probabilities are", wavefunction.get_outcome_probs())
print("This state looks unchanged.\n")

p = Program(H(0))
wavefunction = wavefunction_simulator.wavefunction(p)
print("H|0> = ", wavefunction)
print("The outcome probabilities are", wavefunction.get_outcome_probs())
print("The qubit is in a superposition.")

X|0> =  (1+0j)|1>
The outcome probabilities are {'0': 0.0, '1': 1.0}
This looks like a bit flip.

Y|0> =  1j|1>
The outcome probabilities are {'0': 0.0, '1': 1.0}
This also looks like a bit flip.

Z|0> =  (1+0j)|0>
The outcome probabilities are {'0': 1.0, '1': 0.0}
This state looks unchanged.

H|0> =  (0.7071067812+0j)|0> + (0.7071067812+0j)|1>
The outcome probabilities are {'0': 0.4999999999999999, '1': 0.4999999999999999}
The qubit is in a superposition.


# The Quantum Abstract Machine

We now have enough background to introduce the programming model that underlies Quil. This is a hybrid quantum-classical model in which \(N\) qubits interact with \(M\) classical bits.

These qubits and classical bits come with a defined gate set, e.g. which gate operations can be applied to which qubits. Different kinds of quantum computing hardware place different limitations on what gates can be applied, and the fixed gate set represents these limitations.

The next section on measurements will describe the interaction between the classical and quantum parts of a Quantum Abstract Machine (QAM).

### Qubit Measurements

Measurements have two effects:

1. They project the state vector onto one of the basic outcomes
2. (optional) They store the outcome of the measurement in a classical bit.

Here's a simple example:

In [47]:
# Create a program that stores the outcome of measuring qubit #0 into classical register [0]
p = Program()
classical_register = p.declare('ro', 'BIT', 1)
p += Program(I(0)).measure(0, classical_register[0])

Up until this point we have used the quantum simulator to cheat a little bit --- we have actually looked at the wavefunction that comes back. However, on real quantum hardware, we are unable to directly look at the wavefunction. Instead we only have access to the classical bits that are affected by measurements. This functionality is emulated by `QuantumComputer.run`. Note that the run command is to be applied on the compiled version of the program.


In [48]:
from pyquil import get_qc


qc = get_qc('9q-square-qvm')
print (qc.run(qc.compile(p)))


[[0]]


We see that the classical register reports a value of zero. However, if we had flipped the qubit before measurement then we obtain:

In [50]:
p = Program()
classical_register = p.declare('ro', 'BIT', 1)
p += Program(X(0))   # Flip the qubit
p.measure(0, classical_register[0])   # Measure the qubit

print (qc.run(qc.compile(p)))

[[1]]


These measurements are deterministic, e.g. if we make them multiple times then we always get the same outcome:



In [51]:
p = Program()
classical_register = p.declare('ro', 'BIT', 1)
p += Program(X(0))   # Flip the qubit
p.measure(0, classical_register[0])   # Measure the qubit

trials = 10
p.wrap_in_numshots_loop(shots=trials)

print (qc.run(qc.compile(p)))

[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


### Classical/Quantum Interaction

However this is not the case in general --- measurements can affect the quantum state as well. In fact, measurements act like projections onto the outcome basis states. We can show this using the Hadamard Gate(H). 

A qubit in this state will be measured half of the time in the |0> state, and half of the time in the |1> state. In a sense, this qubit truly is a random variable representing a coin. In fact, there are many wavefunctions that will give this same operational outcome. 

Being able to work with all of these different new states is part of what gives quantum computing extra power over regular bits.



In [57]:
p = Program()
ro = p.declare('ro', 'BIT', 1)

p += Program(H(0)).measure(0, ro[0])

# Measure qubit #0 a number of times
p.wrap_in_numshots_loop(shots=10)

# We see probabilistic results of about half 1's and half 0's
print (qc.run(qc.compile(p)))


[[0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]]


#### PyQuil allows us to look at the wavefunction after a measurement as well:

In [58]:
coin_program = Program(H(0))
print ("Before measurement: H|0> = ", wavefunction_simulator.wavefunction(coin_program), "\n")

ro = coin_program.declare('ro', 'BIT', 1)
coin_program.measure(0, ro[0])
for _ in range(5):
    print ("After measurement: ", wavefunction_simulator.wavefunction(coin_program))


Before measurement: H|0> =  (0.7071067812+0j)|0> + (0.7071067812+0j)|1> 

After measurement:  (1+0j)|1>
After measurement:  (1+0j)|0>
After measurement:  (1+0j)|0>
After measurement:  (1+0j)|0>
After measurement:  (1+0j)|1>


We can clearly see that measurement has an effect on the quantum state independent of what is stored classically. We begin in a state that has a 50-50 probability of being |0> or |1>. After measurement, the state changes into being entirely in |0> or entirely in |1> according to which outcome was obtained. This is the phenomenon referred to as the collapse of the wavefunction. Mathematically, the wavefunction is being projected onto the vector of the obtained outcome and subsequently rescaled to unit norm.

In [59]:
# This happens with bigger systems too, as can be seen with this program,
# which prepares something called a Bell state (a special kind of "entangled state")
bell_program = Program(H(0), CNOT(0, 1))
wavefunction = wavefunction_simulator.wavefunction(bell_program)
print("Before measurement: Bell state = ", wavefunction, "\n")

classical_regs = bell_program.declare('ro', 'BIT', 2)
bell_program.measure(0, classical_regs[0]).measure(1, classical_regs[1])

for _ in range(5):
    wavefunction = wavefunction_simulator.wavefunction(bell_program)
    print("After measurement: ", wavefunction.get_outcome_probs())

Before measurement: Bell state =  (0.7071067812+0j)|00> + (0.7071067812+0j)|11> 

After measurement:  {'00': 0.9999999999999996, '01': 0.0, '10': 0.0, '11': 0.0}
After measurement:  {'00': 0.9999999999999996, '01': 0.0, '10': 0.0, '11': 0.0}
After measurement:  {'00': 0.9999999999999996, '01': 0.0, '10': 0.0, '11': 0.0}
After measurement:  {'00': 0.9999999999999996, '01': 0.0, '10': 0.0, '11': 0.0}
After measurement:  {'00': 0.9999999999999996, '01': 0.0, '10': 0.0, '11': 0.0}


The above program prepares entanglement because, even though there are random outcomes, after every measurement both qubits are in the same state. They are either both |0> or both |1>. This special kind of correlation is part of what makes quantum mechanics so unique and powerful.



### Classical Control

There are also ways of introducing classical control of quantum programs. For example, we can use the state of classical bits to determine what quantum operations to run.

In [60]:
true_branch = Program(X(7)) # if branch
false_branch = Program(I(7)) # else branch

# Branch on ro[1]
p = Program()
ro = p.declare('ro', 'BIT', 8)
p += Program(X(0)).measure(0, ro[1]).if_then(ro[1], true_branch, false_branch)

# Measure qubit #7 into ro[7]
p.measure(7, ro[7])

# Run and check register [7]
print (qc.run(qc.compile(p)))


[[1 1]]


The second [1] here means that qubit 7 was indeed flipped.